# Capstone Project Part II

We need to scrape data from a Wikipedia page, store the data in a dataframe, and clean the data.

In [1]:
import pandas as pd
import numpy as np
import requests
import lxml.html as lh

First we pull the html data from the wiki page by passing the URL to requests. Then we use lxml.html to extract all of the data from the table.

In [2]:
# URL of wiki page with Toronto neighborhood info
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Load webpage
wiki = requests.get(wiki_url)

# Store contents of webpage
wiki_contents = lh.fromstring(wiki.content)

# Extract content of table
table_contents = wiki_contents.xpath('//tr')

Now we organize the table contents and extract the column names from the first row

In [3]:
# Get the column names from the first row of the table
columns = []
for T in table_contents[0]:
    columns.append(T.text_content())
    
print(columns)

['Postal Code\n', 'Borough\n', 'Neighborhood\n']


In [4]:
# Strip \n from the column names
columns = [T.strip() for T in columns]
columns

['Postal Code', 'Borough', 'Neighborhood']

In [5]:
# Load data from table
col_data = []
for i in range(len(columns)):
    col_data.append([])

for i in range(1,len(table_contents)):
    row = table_contents[i]
    
    # Break loop if the length of the row is not 3
    if len(row) != 3:
        break
    
    # Add data to col_data and strip off \n from each string
    for j, T in enumerate(row):
        col_data[j].append(T.text_content().strip())

We can now create a dataframe from the organized data and drop the rows where the borough is not assigned.

In [22]:
# Create dataframe and view head
df = pd.DataFrame({columns[i]: col_data[i] for i in range(len(columns))})
df.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [23]:
# Drop rows where borough is not assigned
df = df[df['Borough'] != 'Not assigned']
df.reset_index(drop = True, inplace = True)
df.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [24]:
df.shape

(104, 3)

Now we download the provided csv containing lat/long data for each of the postal codes and merge it with our dataframe. Note: it's easier to use the link provided in the assignment than a command line command because the link doesn't go directly to the file (it redirects).

In [31]:
!curl -o geospatial.csv https://public.boxcloud.com/d/1/b1!Ao_idQjCstHmIU4YZ2plsN0g3UGE28nxdDm-yiqveeCDewVzjmEhvCqhPVG_1uhwr9bG9Bq80u48Ro2sqhZEKFTfnAVgmtf-GFjLU6DZd07Ftd1HHQOfgdSih4VppMDZd_2s2Y0VV8pTEY__E4RuaKG2vxRjQC3GP_Ergusm3SDN_7XMG5WS7Y6zna0Mzez34sYO8e1r8rHwN7tlpczH88Gj6oG_5RO-zc6I0jdDOtjHYv2QEcrZf6Tq944V06PrcNQM3xn6C7hI-RqiWFAxo5AhAsvww9QN5cpC2JgBnK3MnS98i-V4tKR1EPsG9mgpgeGuk9RzTV3phzeCFUm6TogQokPSYGrfqU2d7z8IaK61sPhnquuTl9XigkgdbwBikuaM4Ty2W1JzEsdJv_rr2TO4YX5JxWWr9gbf-ZbwyB9y3Zf1HokSOSPUHv6-MZLUqR81AhjSCY-PTQ4gu6sMD15wMs5Wf-oI8bARSsge5ORQgkMg1hS7TcEnkKx6OL1nT0RXIOa2U-cE5_2Ljt0ro2CRbpiSflWAcVImI_LvE86YkAL_CACwWQ9B4HjHWjrh5LlPWZs1-m05x7jAIqVv7WkfBlkvSzqOYTFM_m5uSToDdRiOodW5JJKagknPY05iriMrHD6-464TJZMhaAT3jsmPewlhbPdTSaGw03jBD-K8QIFFHiFQMRfct3qjGb00zTfHLZ-xdk3lx-B4F34Eapotbn04RMdRPeJ5OHCVRY3gqGRbaAHNmDvwn60CU-wnENdzoR0XkLybd4cWOUuoYBxHbJRxxdMPKrje5buL4KAdRFp9mUX41xjPXqB-DUROCXxNhyGIJ-mi4ulICPGvY1FWw1p7rHyN4kQ62Lko9UmnAKVgBgcMws7tMwANEwdFDLOgjWUUNTfQktyuGmY4kiE4CZ9-vujZ7r8GFGfD2RAYt93k6lePgrUVMd4_BrLHzv_8QuWhsoYM2NXyRwBihF28oDu6-CgJfhSVloWiYILHuFErV5eAdxGs6PN1Hx_maiMXdeY9Plab7N88FzJ1qBuvDm0Puu_F1VBH582YAHIpi0hOKKTO3g_qmk1F8QhCDXhDbjP_oPQNv1hJIxeJQ1wQwVu3YCKQRch_5xoea1SlOpGZgtOoAOIxZ2ZhIYNhrLcPDr6BTZPGdphyiGtUaXSlWa_bPB8ZXwhW6rMR2cftHnnoak42w-HWoVu4skl893qEbTHPkHBxHykQqntbbHcVS7nRFUdEpgXt5t2MHpgh93V5kqhjqDsuERVRIBm6C1MRZXi784w7k5fF2RL0kwn7ZXHiaaFerjkzHZKW9MGTJ751UiMBpj-SsLrnSJYGMhxIGCpXTAKkyCYEgmpccU8fWWJI5hhp2W1lA0JrbtsVjPf2ZLu7YaLIy4YUSIwS1aZfwEpXzfTn0jg6p73406t5sy737RwRoqmNpUNmNpCEA0I./download
print('Download complete')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  2891  100  2891    0     0   7942      0 --:--:-- --:--:-- --:--:--  7942
Download complete


In [33]:
# Load csv into the notebook and view the head
geo_data = pd.read_csv('geospatial.csv')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [34]:
# Merge lat/long data with df
df = df.merge(geo_data, on = 'Postal Code')
df.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
